# Scan of  Fits flats Files of Monocam
================================================================

- Date of creation : Sunday 5th 2016
- Author : Sylvie Dagoret-Campagne
- Affiliation : LAL/IN2P3/CNRS

In [60]:
import numpy as np

# Set up matplotlib and use a nicer set of plot parameters
#config InlineBackend.rc = {}
#config Backend.rc = {}
import matplotlib
matplotlib.rc_file("../templates/matplotlibrc")  # default config obtained from astropy examples
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [61]:
from astropy.utils.data import download_file
from astropy.io import fits
from astropy import units as u
import os
import ccdproc

In [62]:
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone
import pytz

## 1) Configuration of input files

### 1.1) The input flat files

In [63]:
# flats at the beginning
path='/Users/dagoret-campagnesylvie/iraf/MonoCamMay2016/20160509/projector'
root_filename='gratscan_'
ext_filename='.fits'
startnum=1
#stopnum=6
stopnum=5  # only 30 seconds exposure
date=datetime(2016,5,9)  # 2016-05-10T01:07:55.503
nb_of_channels=16  # number of CCD amplifier channels corresponding to an image inside the fits file

In [64]:
# flats at the beginning
path='/Users/dagoret-campagnesylvie/iraf/MonoCamMay2016/20160509/projector'
root_filename='gscan_'
ext_filename='.fits'
startnum=1
#stopnum=6
stopnum=12  # only 30 seconds exposure
date=datetime(2016,5,9)  # 2016-05-10T01:07:55.503
nb_of_channels=16  # number of CCD amplifier channels corresponding to an image inside the fits file

In [65]:
# the bias master file
masterbias_filename='masterbias1.fits'
masterbias_file=os.path.join(path,masterbias_filename)

In [66]:
def Buildfilename(path,name,ext,num):
    '''
    input:
       path : path of bias files
       name : common root of bias filenames
       ext  : extension of filenames
    output:
       full filename 
    '''
    strnum= '{0:02d}'.format(num)  # python >= 2.6
    filename=name+strnum+ext
    fullfilename=os.path.join(path,filename)
    return fullfilename

In [67]:
filelist= [Buildfilename(path,root_filename,ext_filename,num) for num in range(startnum,stopnum+1,1)]

In [68]:
nbfiles=len(filelist)

### 1.2) configuration of date

In [69]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-06-05 21:40:12.707514
fits date format for the analysis :  2016-06-05T21:40:12


## 2) Exposure

- compute the mean and RMS of the darks for all the pixels

In [70]:
NBCHAN=16
NBFILES=nbfiles

# arrays initialisation
means0=np.zeros((NBCHAN,NBFILES),dtype=np.float)
rms0=np.zeros((NBCHAN,NBFILES),dtype=np.float)
means=np.zeros((NBCHAN,NBFILES),dtype=np.float)
rms=np.zeros((NBCHAN,NBFILES),dtype=np.float)

list_exposures=np.zeros(NBFILES,dtype=np.float)

# loop on files
for file in range(0,nbfiles):
    hdu_list = fits.open(filelist[file])
    header=hdu_list[0].header
    number_of_channels=header['CHAN']
    exposure=header['EXPOSURE']
    list_exposures[file]=exposure
    # loop on channels
    for numchan in range(1,number_of_channels+1):
        image_data = hdu_list[numchan].data
        nrow,ncol =image_data.shape
        # extract the two quatities
        firstcol = image_data[:,0]
        othercol = image_data[:,1:ncol]
        # flatten the arrays
        lowvalue = firstcol.flatten()
        highvalue = othercol.flatten()
        # extract mean and rms
        means0[numchan-1,file]=lowvalue.mean()
        rms0[numchan-1,file]=lowvalue.std()
        means[numchan-1,file]=highvalue.mean()
        rms[numchan-1,file]=highvalue.std()        
                

IndexError: list index out of range

### 2.1) Show exposure versus time

-exposure recorded in fits header file

In [ ]:
title='Skyflat exposure '
fig = plt.figure(figsize=(10,5))
plt.plot(range(NBFILES),list_exposures)    
plt.title(title)
plt.xlabel('skyflats file index')
plt.ylabel('exposure time in sec')
plt.grid(True)

### 2.2) skyflat versus time

In [ ]:
title='Skyflat versus file index '
fig = plt.figure(figsize=(10,10))
for numchan in range(0,number_of_channels):
    plt.errorbar(range(NBFILES),means[numchan,:],yerr=rms[numchan,:])
    
plt.title(title)
plt.xlabel('skyflat index')
plt.ylabel('skyflat ADU')
plt.grid(True)

### 2.3) Skyflats versus index in each amplifier

In [ ]:
NX=4 # number of images along the horizontal axis
NY=4 # number of images along the vertical axis
f, axarr = plt.subplots(NY,NX,figsize=(20,20)) # figure organisation
#f, axarr = plt.subplots(NX,NY,sharex=True, sharey=True,figsize=(20,20))
f.subplots_adjust(hspace=0.5,wspace=0.5)

for index in range(0,number_of_channels):  
    ix=index%4
    iy=index/4
    numchan=index
    axarr[iy,ix].errorbar(range(NBFILES),means[numchan,:],yerr=rms[numchan,:],fmt='o')
    plottitle='channel {}'.format(index+1)
    axarr[iy,ix].set_title(plottitle)
    axarr[iy,ix].set_ylabel('ADU')
    axarr[iy,ix].set_xlabel('dark index')
    axarr[iy,ix].grid(True)

title='ADU vs skyflat index'
plt.suptitle(title,size=16)
plt.show()


### 2.4) skyflat In Biassec

In [ ]:
title='skyflat versus file index first column'
fig = plt.figure(figsize=(10,10))
for numchan in range(0,number_of_channels):
    plt.errorbar(range(NBFILES),means0[numchan,:],yerr=rms0[numchan,:])
    
plt.title(title)
plt.xlabel('skyflat index')
plt.ylabel('ADU')
plt.grid(True)

In [ ]:
NX=4 # number of images along the horizontal axis
NY=4 # number of images along the vertical axis
f, axarr = plt.subplots(NY,NX,figsize=(20,20)) # figure organisation
#f, axarr = plt.subplots(NX,NY,sharex=True, sharey=True,figsize=(20,20))
f.subplots_adjust(hspace=0.5,wspace=0.5)

for index in range(0,number_of_channels):  
    ix=index%4
    iy=index/4
        
    axarr[iy,ix].errorbar(range(NBFILES),means0[numchan,:],yerr=rms0[numchan,:],fmt='o')
    plottitle='channel {}'.format(index)
    axarr[iy,ix].set_title(plottitle)
    axarr[iy,ix].set_ylabel('ADU')
    axarr[iy,ix].set_xlabel('skyflat index')
    axarr[iy,ix].grid(True)
    
title='ADU vs skyflat index'
plt.suptitle(title,size=16)
plt.show()

## 3) Substraction of Master Bias to the Skyflat

### 3.1 Read All Master-Biases

In [ ]:
masterbiases = [ ccdproc.CCDData.read(masterbias_file, hdu=chan,unit="adu") for chan in range(1,nb_of_channels+1,1) ]

### 3.2 Put all darks inside a list of CCDDATA list

In [ ]:
# all CCDPROC data collector : each channel as a list of dark data
allskyflats = []

In [ ]:
for chan in range(1,nb_of_channels+1,1):
    ccd_chan = [ ccdproc.CCDData.read(image_file, hdu=chan,unit="adu") for image_file in filelist ]
    allskyflats.append(ccd_chan)

### 3.3 Substract the masterbias to the Darks 

In [ ]:
mbias_and_flats=zip(masterbiases, allskyflats)

In [ ]:
list_subskyflats= []  # list of list of substracted darks, one list per channel
for mb, sflats in mbias_and_flats:
    list_sf=[]      # list of substracted darks for the current channel
    for sf in sflats:
        s=ccdproc.subtract_bias(sf,mb)
        list_sf.append(s)
    list_subskyflats.append(list_sf)   

## 4) Show the Sky Flat versus Exposure

### 4.1) Compute the average dark

In [ ]:
# book the arrays in advance

newmeans=np.zeros((NBCHAN,NBFILES),dtype=np.float)
newrms=np.zeros((NBCHAN,NBFILES),dtype=np.float)
newexpo=np.zeros((NBCHAN,NBFILES),dtype=np.float)

for fileindex in range(0,nbfiles):
    Xpos=list_exposures[fileindex]
    for channel in range(0,nb_of_channels):       
        newmeans[channel,fileindex]=list_subskyflats[channel][fileindex].data.mean()
        newrms[channel,fileindex]=list_subskyflats[channel][fileindex].data.std()
        newexpo[channel,fileindex]=Xpos

### 4.2) Plot Skyflat vs Exposure for each channels

In [ ]:
NX=4 # number of images along the horizontal axis
NY=4 # number of images along the vertical axis
f, axarr = plt.subplots(NY,NX,figsize=(20,20)) # figure organisation
#f, axarr = plt.subplots(NX,NY,sharex=True, sharey=True,figsize=(20,20))
f.subplots_adjust(hspace=0.5,wspace=0.5)

for index in range(0,number_of_channels):  
    ix=index%4
    iy=index/4
    numchan=index
    axarr[iy,ix].scatter(newexpo[numchan,:],newmeans[numchan,:],s=50,marker='o')
    plottitle='channel {}'.format(index+1)
    axarr[iy,ix].set_title(plottitle)
    axarr[iy,ix].set_ylabel('ADU')
    axarr[iy,ix].set_xlabel('Exposure')
    axarr[iy,ix].grid(True)

title='Skyflat corrected from bias'
plt.suptitle(title,size=16)
plt.show()

### 4.3) Plot skyflat vs Exposure all channels 

In [ ]:
NX=1 # number of images along the horizontal axis
NY=1 # number of images along the vertical axis
f, axarr = plt.subplots(NY,NX,figsize=(10,10)) # figure organisation

for index in range(0,number_of_channels):      
    numchan=index
    lab='channel {}'.format(index+1)
    plt.plot(newexpo[numchan,:],newmeans[numchan,:],'o',label=lab)
    
plt.ylabel('ADU')
plt.xlabel('Exposure')
plt.grid(True)
plt.legend(loc='best')

title='skyflat corrected from bias'
plt.suptitle(title,size=16)
plt.show()